# Sprungantwort einer PTn-Strecke

In [32]:

import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as w
import numpy as np
import scipy as sc
import control

from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

# animate plolspace?
animate=True # True / False

K=1
response=([],[])

@interact(T1=(1, 10), T2=(1, 10), T3=(1, 10), U_SPRUNG=(-3,3,0.1), T_FINAL=(20,100,5), U_START=(2,20,1), K_PS=(1, 3, 0.1))
def f(T1=2, T2=3, T3=5, T_SPRUNG=3, U_SPRUNG=2,  T_FINAL=40, U_START=10, K_PS=2):
    global response
    den=np.polymul(np.polymul([T1, 1], [T2, 1]), [T3, 1])
    T_vec=np.arange(0, T_FINAL, 0.1)
    U_vec=[0,]*10*T_SPRUNG + [U_SPRUNG,]*((T_FINAL-T_SPRUNG)*10)
    plant = control.TransferFunction([K_PS], den)
    response = control.forced_response(plant, T_vec, U_vec)
    response.y=response.y+U_START
    #und jetzt wird geplottet
    fig, ax = plt.subplots(figsize=(10 , 5))

    ax.plot(response[0], response[1])
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(1))
    ax.set_ylim([U_START-2, U_START+(K_PS*U_SPRUNG)+2])
    ax.set_xlim(T_SPRUNG-2, T_FINAL)
    
    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.grid(which='major', color='#DDDDDD')
    ax.grid(which='minor', color='#EEEEEE', linestyle=':')

    plt.show()
    return;


interactive(children=(IntSlider(value=2, description='T1', max=10, min=1), IntSlider(value=3, description='T2'…

In [47]:
import openpyxl as xls
import os
def btnExport_on_click(b):
       wb = xls.Workbook()
btnExport = w.Button(description='Export to Excel')
btnExport.on_click(btnExport_on_click)
display(btnExport)

wb = xls.Workbook()
ws = wb.active
ws.append(["Time", "Response"])
for i in range(len(response.t)):
       ws.append([response.t[i], response.y[0][i]])
chart = xls.chart.ScatterChart()
chart.style = 2
chart.x_axis.title = 'Zeit [s]'
chart.y_axis.title = 'y [1]'
xvalues = xls.chart.Reference(ws, min_col=1, min_row=2, max_row=1+len(response.t))
values  = xls.chart.Reference(ws, min_col=2, min_row=1, max_row=1+len(response.t))
series = xls.chart.Series(values, xvalues, title_from_data=True)
chart.series.append(series)

minorGridlines = xls.chart.shapes.GraphicalProperties() 
minorGridlines.line.solidFill='0070C0'
minorGridlines.line.dashStyle="dash"

chart.x_axis.scaling.min = 0
chart.x_axis.scaling.max = max(response.t)
chart.y_axis.scaling.min = min(response.y[0])-2
chart.y_axis.scaling.max = max(response.y[0])+2

chart.x_axis.majorUnit=5
chart.x_axis.majorTickMark='out'

chart.x_axis.minorUnit=1
chart.x_axis.minorTickMark='in'
chart.x_axis.minorGridlines=xls.chart.axis.ChartLines(spPr=minorGridlines)


chart.y_axis.majorUnit=5
chart.y_axis.minorUnit=1
chart.y_axis.minorGridlines=xls.chart.axis.ChartLines(spPr=minorGridlines)

chart.title = ""
ws.add_chart(chart, "D1")
wb.save("sprungantwort.xlsx")

Button(description='Export to Excel', style=ButtonStyle())